In [8]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import pad_sequences
from keras_preprocessing.text import Tokenizer
import os
import numpy as np
import pickle
from keras import mixed_precision
import re

In [9]:
# GPU Settings
print(tf.config.list_physical_devices('GPU'))

# Conda activate tf-gpu to active the GPU envrioment

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)
else:
    print("Bruh")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
# Dataset Preparation

input_texts = []
output_texts = []

with open("Login_Form_Dataset_2_5m/code.txt") as f:
  code_text = f.read()

card_blocks = re.findall(r'(<Card[\s\S]*?</Card>)', code_text)

output_texts = card_blocks

with open("Login_Form_Dataset_2_5m/prompts.txt")as f :
  input_texts = f.read().splitlines()





In [12]:
# Tokenizer
tokenizer = Tokenizer(num_words= 2000, oov_token="<OOV>", char_level=False,split=" ",lower=False,filters="")

# So the Tokenizer converts words to a Token ID
# We have two Tokenizers one for Input taking the Prompt in this Case: "Button, blue-500, round, "Click me" "
# And we have one for Outputing the Answer of the Model in this case <Button classname="" ... > and so on
# num words is the ammount of the Top words we keep for the Tokenizer

combined_texts = []
for prompt, code in zip(input_texts, output_texts):
    combined_text = "<Start> " + prompt + " <SEP> " + code + " <End>"
    combined_texts.append(combined_text)

tokenizer.fit_on_texts(combined_texts)
tokenizer.word_index = {"<PAD>": 0, **{k: v+1 for k, v in tokenizer.word_index.items()}}
tokenizer.index_word = {v: k for k, v in tokenizer.word_index.items()}

print("Total unique tokens in dataset:", len(tokenizer.word_index))  

# Training the Tokenizer on the vocab and adding Start and End Tokens
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
    
sequences = tokenizer.texts_to_sequences(combined_texts)


# So the text_to_sequences is therefore to convert each word or also called Token to its corresponding ID
# In this Case we first train the Tokenizers to generate IDs, and than we let the Tokenizers generate IDs

max_seq_len = max(len(seq) for seq in sequences)

# These are the neccessary lengths so we can pad all sequences in a batch to have the same size this process is calling padding 
# So we get the max length and afterwards check if the length of sequence is long enough else we add zero's till the max_encoder / max_decoder length is reached

sequences = pad_sequences(sequences,maxlen=max_seq_len,padding='post',truncating="post")
print(max_seq_len)

vocab_size = len(tokenizer.word_index) + 1


Total unique tokens in dataset: 1162


KeyboardInterrupt: 

In [ ]:
embed_dim = 384     
num_heads = 12          
ff_dim = 1536       
num_layers = 6



inputs = layers.Input(shape=(max_seq_len-1))

embeddinglayer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim,mask_zero=True,name="embedding_layer")(inputs)

positional_encoding = layers.Embedding(input_dim=max_seq_len-1,output_dim=embed_dim,name="positional_encoding_layer")(tf.range(start=0, limit=max_seq_len-1,delta=1))

x = embeddinglayer + positional_encoding

# Only Self attention no encoder decoder attention
for i in range(num_layers):
  self_attention = layers.MultiHeadAttention(
  num_heads = num_heads,
  key_dim= embed_dim // num_heads,
  dropout=0.1,
  name=f"masked_self_attention_{i}"
  )(
    query=x,
    value=x,
    key=x,
    use_causal_mask=True   
  )
    
  x1 = layers.LayerNormalization(epsilon=1e-6)(x + self_attention) # residual connection by addiding the attention scores to the previous ones plus semantic context

  ffn = tf.keras.Sequential([
    layers.Dense(ff_dim,activation="gelu"),
    layers.Dropout(0.1),
    layers.Dense(embed_dim)
  ], name=f"feed_forward_{i}")

  ffn_output = ffn(x1)

  x = layers.LayerNormalization(epsilon=1e-6)(ffn_output + x1)

outputs = layers.Dense(
  vocab_size,
  activation="softmax",
  name="decoder_output_dense"
)(x)


In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs, name="decoder_only_transformer")

initial_learning_rate = 1e-4
lr_schedule = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=1000,
    alpha=0.1
)



model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()


callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,  
        restore_best_weights=True,
        verbose=1
    ),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    keras.callbacks.ModelCheckpoint(
        filepath='Models/LoginFormModelNew.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    
    keras.callbacks.LambdaCallback(
        on_batch_end=lambda batch, logs: tf.clip_by_global_norm([v for v in model.trainable_variables], 1.0)
    )
]


X = sequences[:, :-1]  
y = sequences[:, 1:]   


batch_size = 32
validation_split = 0.2 

history = model.fit(
    X,
    y,
    batch_size=batch_size,
    epochs=1,
    validation_split=validation_split,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

model.save("Models/LoginFormModelNew.keras")


Model: "decoder_only_transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 164)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 164, 384)     446976      ['input_1[0][0]']                
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 164, 384)    0           ['embedding_layer[0][0]']        
 da)                                                                                              
                                                                                                  
 masked_self_attention_0 (Multi  (None, 164, 384)    591360      ['tf.__ope

In [ ]:
# Test Model

decoder_transformer = keras.models.load_model('Models/LoginFormModel.keras')
def generate_button_code(prompt):
    prompt = "<Start> " + prompt + " <SEP>"
    
    input_sequence = tokenizer.texts_to_sequences([prompt])[0]
    current_sequence = pad_sequences([input_sequence], maxlen=max_seq_len-1, padding='post')
    
    generated_tokens = []
    for _ in range(200):  
        predictions = decoder_transformer.predict(current_sequence, verbose=0)
        next_token = np.argmax(predictions[0, len(input_sequence) + len(generated_tokens) - 1])
        
        if next_token == tokenizer.word_index.get("<End>", 0) or next_token == 0:
            break
            
        generated_tokens.append(next_token)
        
        new_sequence = input_sequence + generated_tokens
        current_sequence = pad_sequences([new_sequence], maxlen=max_seq_len-1, padding='post')
    
    result = [tokenizer.index_word.get(token, "") for token in generated_tokens]
    return " ".join(result)

test_prompt = "Build a user sign in component with social login options"
generated = generate_button_code(test_prompt)
print("Generated Button Code:", generated)

ValueError: in user code:

    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\input_spec.py", line 295

# Notes

- first Padding with Tokenizer map it 0 good Fix 
- Decoder Only model is very powerful, but if too big == no good results
- Dataset Quality is the second Important Part

# Further Improvements

- Finetuning the Model on specific Problems 
- More Data with more variation esspecially with the Prompt input


# Next Steps

- One Model for everything 
- with better login Form Dataset and better GPU optimization 